In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

Read the data

In [53]:
coin_vals = np.array([5, 10, 20, 50, 100, 200, 500])

X = []
y = []

for val in coin_vals:
    path = os.path.join('data', str(val))
    for file in os.listdir(path):
        if file.endswith(".png"):
            imgdata = plt.imread(os.path.join(path, file))[0:100, 0:100, 0]
            X.append(imgdata)
            y.append(coin_vals == val)

X = np.array(X)
y = np.array(y, dtype=np.float32)


Create our model

In [77]:
sess = tf.Session()
with sess:
    x = tf.placeholder(tf.float32, [None, 100, 100, 1])
    y_ = tf.placeholder(tf.float32, [None, 7])
    
    W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 8], stddev=0.3))
    b1 = tf.Variable(tf.constant(0.3 * np.ones([50, 50, 8]), dtype=tf.float32))
    
    W2 = tf.Variable(tf.truncated_normal([3, 3, 8, 16], stddev=0.3))
    b2 = tf.Variable(tf.constant(0.3 * np.ones([25, 25, 16]), dtype=tf.float32))
    
    W3 = tf.Variable(tf.truncated_normal([12, 7], stddev=0.3))
    b3 = tf.Variable(tf.constant(0.3 * np.ones([7]), dtype=tf.float32))
    
    def pipeline(x):
        h1 = x
        h1 = tf.nn.max_pool(h1, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
        h1 = tf.nn.conv2d(h1, W1, [1, 1, 1, 1], 'SAME') + b1
        h1 = tf.nn.relu(h1)
        
        h2 = tf.nn.max_pool(h1, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
        h2 = tf.nn.conv2d(h2, W2, [1, 1, 1, 1], 'SAME') + b2
        h2 = tf.nn.relu(h2)
        
        out = tf.nn.max_pool(h2, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
        out = tf.reshape(out, [-1, 12])
        out = tf.matmul(out, W3) + b3
        
        return out
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=pipeline(x)))
    train_task = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
    sess.run(tf.global_variables_initializer())